### Note:
In this directory, there are two examples using PDFs: **Example 3 - Plot PDF for a station**, and **Example 4 - Calculate PDFs from PSDs**.  These two examples are provided in order to highlight different ways that you can use the PDF and PSD values that ISPAQ generates.  

To be specific, the difference between the two examples are:  

#### Example 3 - Plot PDF for a station:  
Example 3 uses PDFs that already exist in the ISPAQ metric database. This means that they have been calculated using an ispaq.py command with the `--output db` option.  

This is a great way to do it, especially if you plan to run the PSDs and PDFs at the same time, say on some sort of regular schedule.  In that case, you might as well calculate both in the same command and store them both in the ISPAQ database for later retrieval.  

Additionally, we have tried to make it simple to calculate PDFs in ISPAQ for cases where you already have PSDs for the time span you are interested in.  For example, PDFs calculation does not require seismic data since it instead reads in existing PSDs.  That means that if you, the user, have been calculating daily PSDs for the past year, you don’t need to load a year’s worth of data to calculate a year-long PDF  - you can just use the existing PSDs! By calculating that year-long PDF using ISPAQ, it will be saved to either the database or the csv file and you will be able to retrieve it later.  


#### Example 4 - Calculate PDFs from PSDs:  
Example 4 will calculate PDFs _on the fly_, meaning that they do not need to exist in the ISPAQ metric database, nor will they be saved to the ISPAQ metric database.   

Why would you want to do this if you can simply use an ispaq.py command to calculate and save the PDFs in the database?  Here are a couple possible reasons: 
1) You may want to calculate PDFs on an arbitrary timeframe but don't feel the need to save the PDF values, say if you are just poking around at or investigating changes in the data and don't want to clutter the database.   

2) To prevent the ISPAQ database from growing too complicated, the pdf table in the ISPAQ database is very simple and PDFs values are stored with the start and end times used to calculate that particular PDF.  If you calculate daily PDFs for a week and then additionally calculate a week-long PDF, the database will store 8 PDFs - one for each day in the week, and one that spans the entire week. This means that, even if you have used ISPAQ to calculate your arbitrary time frame, you must know the specific start and end times of the PDF that you are looking to retrieve. If you look for a time range using less-than and greater-than (<>) instead of equals-to (==) then you risk retrieving multiple PDFs, including ones that you did not intend. By using this on-the-fly method, you bypass this risk since PSDs are stored by the individual PSD (usually an hour span, can vary depending on the sample rate of the data), and only those PSDs that are needed to calculate the PDF are retrieved. 

  
  
*Both methods are valid and can be useful in different situations.*

## Example 4 - Calculate PDFs from PSDs
The intent of this series of Jupyter Notebooks is to demonstrate how metrics can be retrieved from the sqlite metrics database and provide some ideas on how to use or plot those metrics.  

In this example, we will use the pdf_aggregator.py script to calculate PDFs on-the-fly.  The calculate_pdfs method in the pdf_aggregator will look for existing PSDs, calculate the PDFs, and then return the PDF values in a dataframe.  It also returns dataframes of the modes, minimums, and maximums. 

To generate PDFs in this example, corrected PSD values must already exist. If they do not yet exist, then you can run them via:

    ./run_ispaq.py -M psd_corrected -S ANMO --starttime 2020-10-01 --endtime 2020-10-26 --output db


This example is slightly more complicated because it is tying into the ISPAQ code directly. That means that we need to import multiple functions from various ISPAQ scripts, and we need the proper arguments available for each of those. 

First we import the simple ones:

In [ ]:
import sys
import os
import logging
import pandas as pd
from obspy import UTCDateTime

Now we move onto the ISPAQ-specific ones. Because of the directory structure and where this example lives, we need to add the main ispaq directory to our path. Then we will be able to import the ISPAQ modules. 

In [ ]:
path_parent = os.path.dirname(os.getcwd())
sys.path.insert(1, f'{path_parent}/ispaq/')


import concierge
from user_request import UserRequest
import PDF_aggregator

With the modules imported, we now need to set up some variables that will be required to run the ISPAQ code. This includes a logger and an arguments class that contains the fields from the preference file.

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

class args:
    db_name = 'ispaq.db'
    starttime = UTCDateTime('2020-10-01')
    endtime = UTCDateTime('2021-10-15')
    metrics = 'pdf'
    stations = 'IU.ANMO.00.BHZ.M'  # The "stations" must refer to a single target, including the quality code (N.S.L.C.Q)
    preferences_file = f'{path_parent}/preference_files/default.txt'
    station_url = 'IRIS'
    dataselect_url = 'IRIS'
    event_url = 'IRIS'
    resp_dir = ''
    output = 'db'
    csv_dir = f'{path_parent}/csvs/'
    sncl_format = 'N.S.L.C.'
    sigfigs = 6
    pdf_type = 'plot'
    pdf_interval = 'aggregated'
    plot_include = ''
    pdf_dir = f'{path_parent}/pdfs/'
    psd_dir = f'{path_parent}/psds/'
    


Those will now be used to create a userRequest, which will then be used to create a concierge object. The concierge object will need to later be passed into the method that actually calculates the PDFs.

In [ ]:
user_request = UserRequest(args, logger=logger)
concierge = concierge.Concierge(user_request, logger=logger)
print(concierge, logger)
print(concierge.logger)

Now that we've handled that, we can calculate the PDFs. First, we move into the directory that contains the database, since that's where the ISPAQ code expects us to be. Then we call on the calculate_PDF method, which will return dataframes that contain: PDF values, modes, maximums, and minimums.

In [ ]:
os.chdir(path_parent)
[pdfDF,modesDF, maxDF, minDF] = PDF_aggregator.calculate_PDF(pd.DataFrame(), args.stations, args.starttime, args.endtime, concierge)
print(pdfDF)

With it in a dataframe, you can now do what you want with it! Manipulate it how you want.  

Below I call on the plot_PDF function to plot it up and save the figure to the pdf_dir specified above.

In [ ]:
PDF_aggregator.plot_PDF(args.stations, args.starttime, args.endtime, pdfDF, modesDF, maxDF, minDF, concierge)